# Конвертирует разметку из json-файла в набор xml-файлов для обучения darkflow

In [7]:
img_shape = (1280,720,3) #размер изображения

In [8]:
import json, os

In [9]:
from dict2xml import dict2xml

In [10]:
json_source_path = 'C:\\my_cv_work\\tpe_recognition\\data\\cam_1_hand_markup'
json_file = 'cam_1_hand_markup.json'
images_path = 'D:\\data\\pics\\cam_1_src'

In [11]:
# папка для хранения xml-разметки
xml_path = os.path.join(json_source_path, 'to_xml')
if not os.path.exists(xml_path):
    os.mkdir(xml_path)

In [12]:
# загружаем данные из словаря с разметкой
data = json.loads(open(os.path.join(json_source_path,json_file), 'r').read())

In [23]:
# делаем словарь "чистой" разметки:
ground_truth = { k:v for k,v in data.items() if v['n_boxes'] > 0}

clean_dict = dict()
for k,v in ground_truth.items():
    good_boxes = []
    for bbox in v['bboxes']:
        if (bbox['bottomright'][0]-bbox['topleft'][0])*(bbox['bottomright'][1]-bbox['topleft'][1])>0:
            good_boxes.append(bbox)
    if len(good_boxes)>0:
        clean_dict[k] = {'bboxes': good_boxes}
        
for k,v in clean_dict.items():
    for bbox in v['bboxes']:
        assert (bbox['bottomright'][0]-bbox['topleft'][0])*(bbox['bottomright'][1]-bbox['topleft'][1])>0, 'file: {}'.format(k)
        
print(len(clean_dict))

1425


In [38]:
# делаем xml-файлы
for k,v in clean_dict.items():
    img_filename = os.path.split(k)[-1]
    xml_filename = os.path.splitext(img_filename)[0]+'.xml'
    styled_dict = convert_dict_to_xml_style(img_filename, v, size=img_shape)
    xml_data = dict2xml(styled_dict)
    with open(os.path.join(xml_path, xml_filename), 'w') as f:
        f.write(xml_data)

In [36]:
def convert_dict_to_xml_style(img_filename, bboxes, folder='folder_1', size=None):
    objects = []
    obj_dict = {}
    for bbox in bboxes['bboxes']:
        obj_dict['name'] = 'person'
        obj_dict['pose'] = 'Left'
        obj_dict['truncated'] = 0
        obj_dict['difficult'] = 0
        obj_dict['bndbox'] = {'xmin': bbox['topleft'][0],
                              'ymin': bbox['topleft'][1],
                              'xmax': bbox['bottomright'][0],
                              'ymax': bbox['bottomright'][1]}
        objects.append(obj_dict)
    sizes = {'width': size[0], 'height': size[1], 'depth': size[2]}
    data = {}
    data['folder'] = folder
    data['filename'] = img_filename
    data['size'] = sizes
    data['object'] = objects
    return {'annotation': data}